In [1]:
#https://colab.research.google.com/github/Stable-Baselines-Team/rl-colab-notebooks/blob/sb3/pretraining.ipynb#scrollTo=eI1EFFnW8ao-
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon May 23 15:30:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:06:00.0  On |                  N/A |
| 53%   46C    P3    32W / 200W |   1103MiB /  7981MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from sklearn import tree
from sklearn.tree import export_text
from collections import deque

In [3]:
import gym
from tqdm.notebook import tqdm
import numpy as np
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR, CyclicLR, CosineAnnealingWarmRestarts
import torch.nn.functional as F
import os
import time
import random
import sys
from dagger import get_env_and_model
import torch
from matplotlib import pyplot as plt
import math
import gc
from torch.utils.data.dataset import Dataset, random_split
import pickle

import wandb
from skimage.transform import resize

In [4]:
from IPython import display as ipythondisplay

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
if torch.cuda.is_available:
    device = "cuda:0"
    print('Using GPU')
else:
    device = "cpu"
    print('using CPU')

Using GPU


In [8]:
import gym
import random

def Random_games():
    for episode in range(10):
        env.reset()
        for t in range(500):
            env.render()
            action = env.action_space.sample()
         

In [11]:
def stack2sym(stack):
    x4 = stack[-1]
    x3 = stack[-2]
    x2 = stack[-3]
    x1 = stack[-4]
    
    #-------------------------------------------
    #--------player section---------------------
    #-------------------------------------------
    
    # get player x,y, cpu positions
    player_x4, player_y4 = x4[70], x4[97]
    player_x3, player_y3 = x3[70], x3[97]
    player_x2, player_y2 = x2[70], x2[97]
    player_x1, player_y1 = x1[70], x1[97]
    
    # get player x velocities
    player_vx1 = player_x2 - player_x1
    player_vx2 = player_x3 - player_x2
    player_vx3 = player_x4 - player_x3
    
    # get player y velocities
    player_vy1 = player_y2 - player_y1
    player_vy2 = player_y3 - player_y2
    player_vy3 = player_y4 - player_y3
    
    # get player x accs
    player_ax1 = (player_vx2 - player_vx1)/2
    player_ax2 = (player_vx3 - player_vx2)/2
    
    # get player y accs
    player_ay1 = (player_vy2 - player_vy1)/2
    player_ay2 = (player_vy3 - player_vy2)/2
    
    # get player jerks 
    player_jx = (player_ax2 - player_ax1)/4
    player_jy = (player_ay2 - player_ay1)/4
    
    # player direction
    player_dir4 = x4[86]
    
    # player missile dir
    player_mdir4 = x4[87]
    
    # player missile x
    player_mx4 = x4[103]
    
    # player oxy
    player_oxy4 = x4[102]
    
    # n divers collected
    n_divers4 = x4[62]
    
    #-------------------------------------------
    #--------enemy section---------------------
    #-------------------------------------------
    enemy_obstacle_x1_4 = x4[30]
    enemy_obstacle_x2_4 = x4[31]
    enemy_obstacle_x3_4 = x4[32]
    enemy_obstacle_x4_4 = x4[33]
    
    enemy_obstacle_x1_3 = x3[30]
    enemy_obstacle_x2_3 = x3[31]
    enemy_obstacle_x3_3 = x3[32]
    enemy_obstacle_x4_3 = x3[33]
    
    enemy_obstacle_vx1_3 = enemy_obstacle_x1_4 - enemy_obstacle_x1_3
    enemy_obstacle_vx2_3 = enemy_obstacle_x2_4 - enemy_obstacle_x2_3
    enemy_obstacle_vx3_3 = enemy_obstacle_x3_4 - enemy_obstacle_x3_3
    enemy_obstacle_vx4_3 = enemy_obstacle_x4_4 - enemy_obstacle_x4_3
    
    # diver or enemy missile x
    doe_m_x1_4 = x4[71]
    doe_m_x2_4 = x4[72]
    doe_m_x3_4 = x4[73]
    doe_m_x4_4 = x4[74]
    
    doe_m_x1_3 = x3[71]
    doe_m_x2_3 = x3[72]
    doe_m_x3_3 = x3[73]
    doe_m_x4_3 = x3[74]
    
    doe_m_vx1_3 = doe_m_x1_4 - doe_m_x1_3
    doe_m_vx2_3 = doe_m_x2_4 - doe_m_x2_3
    doe_m_vx3_3 = doe_m_x3_4 - doe_m_x3_3
    doe_m_vx4_3 = doe_m_x4_4 - doe_m_x4_3
    
    #29
    return np.array([player_x4,
                     player_y4,
                     player_vx3,
                     player_vy3,
                     player_ax2,
                     player_ay2,
                     player_jx,
                     player_jy,
                     player_dir4,
                     player_mdir4,
                     player_mx4,
                     player_oxy4,
                     n_divers4,
                     enemy_obstacle_x1_4,
                     enemy_obstacle_x2_4,
                     enemy_obstacle_x3_4,
                     enemy_obstacle_x4_4,
                     enemy_obstacle_vx1_3,
                     enemy_obstacle_vx2_3,
                     enemy_obstacle_vx3_3,
                     enemy_obstacle_vx4_3,
                     doe_m_x1_4,
                     doe_m_x2_4,
                     doe_m_x3_4,
                     doe_m_x4_4,
                     doe_m_vx1_3,
                     doe_m_vx2_3,
                     doe_m_vx3_3,
                     doe_m_vx4_3]
                   )

In [12]:
env, expert = get_env_and_model(env='SeaquestNoFrameskip-v4')

Loading latest experiment, id=1
filter:  rl-trained-agents
algo:  ppo
env_id:  SeaquestNoFrameskip-v4
exp id:  1
log path:  rl-trained-agents/ppo/SeaquestNoFrameskip-v4_1
hyperparams:  OrderedDict([('batch_size', 256), ('clip_range', 'lin_0.1'), ('ent_coef', 0.01), ('env_wrapper', ['stable_baselines3.common.atari_wrappers.AtariWrapper']), ('frame_stack', 4), ('learning_rate', 'lin_2.5e-4'), ('n_envs', 8), ('n_epochs', 4), ('n_steps', 128), ('n_timesteps', 10000000.0), ('policy', 'CnnPolicy'), ('vf_coef', 0.5), ('normalize', False)])
env kwargs:  {}
Stacking 4 frames
Wrapping the env in a VecTransposeImage.
initial obs shape:  (1, 84, 84, 4)


In [13]:
n_steps = 5
n_data = int(5)
frame_stack = deque(maxlen=4)

state = env.reset()
ram_state = env.envs[0].unwrapped._get_ram()
frame_stack.append(ram_state)

expert_acts = np.empty((n_data))
expert_obs_sym = np.empty((n_data,29))

n = 0
while n < n_data:
    
    correct_action = expert.predict(state, state=state, deterministic=False)
    new_state,rwd,done,_ = env.step(correct_action)
    new_ram_state = env.envs[0].unwrapped._get_ram()
    
    
    frame_stack.append(new_ram_state)
    
    #save
    if len(frame_stack) == 4:
        expert_obs_sym[n] = stack2sym(list(frame_stack))
        print('stack2sym: ', expert_obs_sym[n])
        expert_acts[n] = correct_action[0]
        
        n += 1
    
    state = new_state
    
    if done:
        state = env.reset()

stack2sym:  [76. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0. 24.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
stack2sym:  [76. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0. 26.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
stack2sym:  [76. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0. 28.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
stack2sym:  [76. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0. 30.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
stack2sym:  [76. 13.  0.  0.  0.  0.  0.  0.  0.  0.  0. 32.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [22]:
def flip_fidelity(model,n_eps = 30, fidelity=False):
    env, expert = get_env_and_model(env='SeaquestNoFrameskip-v4')
    '''
    state = env.reset()
    ram_state = env.envs[0].unwrapped._get_ram()
    print('ram state: ', ram_state)
    sym_state = rams2sym_pong([ram_state,ram_state])
    print('sym state: ', sym_state)
    '''
    if True:
        j = 0
        ep_rwd = 0
        steps_survived = 0
        scores = []
        n_corr = 0
        
        flip_correct = 0
        flip_total = 0
        
        expert_actions, agent_actions = [], []
        
        while len(scores) < n_eps:
            if j == 0:
                state = env.reset()
                ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(ram_state)
                steps_survived = 1
                done = False
            else:
                # get correct action
                correct_action =  expert.predict(state, state=state, deterministic=False)
                #print('exp action: ', correct_action[0])
                expert_actions.append(correct_action[0])
                
                # get current agent action
                if len(frame_stack) == 4:
                    sym_state = stack2sym(list(frame_stack))    
                    agent_action = clf.predict(np.expand_dims(sym_state,0))
                    #print('agent act: ', agent_action[0])
                    agent_actions.append(agent_action[0])
                else:
                    agent_action = [0]
                    agent_actions.append(agent_action[0])
                    
                if fidelity:
                    if correct_action[0] == agent_action[0]:
                        n_corr += 1
                    
                # take step
                next_state, rwd, done, _ = env.step(correct_action)
                
                if len(expert_actions) > 1 and expert_actions[-1] != expert_actions[-2]:
                    flip_total += 1
                    if expert_actions[-1] == agent_actions[-1]:
                        flip_correct += 1
                
                # record reward/survival
                ep_rwd += rwd
                steps_survived += 1
                
                # get current ram state
                new_ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(new_ram_state)
                #new_sym_state = stack2sym(list(frame_stack))
                
            
                state = next_state #unindent?
                #sym_state = new_sym_state
            
            if done:
                print('Ep rwd: ', ep_rwd, ' Survived: ', steps_survived)
                scores.append(ep_rwd)
                # clear the deque
                frame_stack.clear()
                
                state = env.reset()
                ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(new_ram_state)
                #sym_state = stack2sym(list(frame_stack))
                
                #n += 1
                ep_rwd = 0
                steps_survived = 0
            j += 1
        
    if fidelity:
        print('Fidelity: ', n_corr / j, 'sensititivty: ', flip_correct / flip_total)

In [15]:
def deploy_clf(model,n_eps = 30, fidelity=False):
    env, expert = get_env_and_model(env='SeaquestNoFrameskip-v4')
    '''
    state = env.reset()
    ram_state = env.envs[0].unwrapped._get_ram()
    print('ram state: ', ram_state)
    sym_state = rams2sym_pong([ram_state,ram_state])
    print('sym state: ', sym_state)
    '''
    if True:
        j = 0
        ep_rwd = 0
        steps_survived = 0
        scores = []
        n_corr = 0
        while len(scores) < n_eps:
            if j == 0:
                state = env.reset()
                ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(ram_state)
                steps_survived = 1
                done = False
            else:
                # get correct action
                correct_action =  expert.predict(state, state=state, deterministic=False)
                
                # get current agent action
                if len(frame_stack) == 4:
                    sym_state = stack2sym(list(frame_stack))    
                    agent_action = clf.predict(np.expand_dims(sym_state,0))

                else:
                    agent_action = [0]
                    
                if fidelity:
                    if correct_action[0] == agent_action[0]:
                        n_corr += 1
                    
                # take step
                if fidelity:
                    next_state, rwd, done, _ = env.step(correct_action)
                else:
                    next_state, rwd, done, _ = env.step(agent_action)
                
                # record reward/survival
                ep_rwd += rwd
                steps_survived += 1
                
                # get current ram state
                new_ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(new_ram_state)
                #new_sym_state = stack2sym(list(frame_stack))
                
            
                state = next_state #unindent?
                #sym_state = new_sym_state
            
            if done:
                print('Ep rwd: ', ep_rwd, ' Survived: ', steps_survived)
                scores.append(ep_rwd)
                # clear the deque
                frame_stack.clear()
                
                state = env.reset()
                ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(new_ram_state)
                #sym_state = stack2sym(list(frame_stack))
                
                #n += 1
                ep_rwd = 0
                steps_survived = 0
            j += 1
    
    print('avg score: ', np.average(scores))
    if fidelity:
        print('Fidelity: ', n_corr / j)

In [16]:
def dagger(n_deploy=1600,n_dagger=20):
    env, expert = get_env_and_model(env='SeaquestNoFrameskip-v4')
    avg_accs, avg_survs, avg_rs = [], [], []
    clf = tree.DecisionTreeClassifier()#max_depth=4)
    
    frame_stack = deque(maxlen=4)
    
    for i in range(n_dagger):
        if i == 0:
            agg_obs = expert_obs_sym
            agg_acts = expert_acts
        
        print('\nDagger iteration ', i)
        print('data size: ', agg_obs.shape[0])
        try:
            #idx = ~np.isnan(agg_obs).any(axis=1)
            #agg_obs = agg_obs[idx]
            #agg_acts= agg_acts[idx]
            print(np.isnan(agg_obs).any(axis=1))
            clf = clf.fit(agg_obs, agg_acts.astype(int))
        except Exception as e:
            print(e)
            print(agg_obs[-1])
            print('fb: ', frame_stack)
            break
        print('benchmark acc: ', clf.score(expert_obs_sym, expert_acts.astype(int)))
        print('agg acc: ', clf.score(agg_obs, agg_acts.astype(int)))
        
        
        # get trajectory from agent
        traj_obs_sym = np.empty((n_deploy,29))
        traj_acts = np.empty(n_deploy)
        
        n = 0
        ep_rwd = 0
        steps_survived = 0
        for j in range(n_deploy+100):
            # check if enough data gathered
            if n == n_deploy:
                break
            
            if j == 0:
                state = env.reset()
                ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(ram_state)
                steps_survived = 1
                done = False
            else:
                # get correct action
                correct_action =  expert.predict(state, state=state, deterministic=False)
                
                # get current agent action
                if len(frame_stack) == 4:
                    sym_state = stack2sym(list(frame_stack))    
                    agent_action = clf.predict(np.expand_dims(sym_state,0))
                    
                    # record state/action from current frame_stack
                    try:
                        if np.isnan(sym_state).any():
                            print('ADDED A NAN')
                        
                        traj_obs_sym[n] = sym_state
                        traj_acts[n] = correct_action[0]
                        n = n + 1
                    except Exception as e:
                        print('ERROR')
                        print(e)
                        break
                    
                    
                    
                else:
                    agent_action = [0]
                    
                # take step
                next_state, rwd, done, _ = env.step(agent_action)
                
                # record reward/survival
                ep_rwd += rwd
                steps_survived += 1
                
                # get current ram state
                new_ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(new_ram_state)
                #new_sym_state = stack2sym(list(frame_stack))
                
            
                state = next_state #unindent?
                #sym_state = new_sym_state
            
            if done:
                print('Ep rwd: ', ep_rwd, ' Survived: ', steps_survived)
                # clear the deque
                frame_stack.clear()
                
                state = env.reset()
                ram_state = env.envs[0].unwrapped._get_ram()
                frame_stack.append(new_ram_state)
                #sym_state = stack2sym(list(frame_stack))
                
                #n += 1
                ep_rwd = 0
                steps_survived = 0
        
        # aggregate new trajectories
        agg_obs, agg_acts = np.vstack((agg_obs, traj_obs_sym)), np.concatenate((agg_acts, traj_acts))
        
    env.close()
                
            
    return clf
            

In [20]:
clf = dagger(n_dagger=60,n_deploy=3200)

Loading latest experiment, id=1
filter:  rl-trained-agents
algo:  ppo
env_id:  SeaquestNoFrameskip-v4
exp id:  1
log path:  rl-trained-agents/ppo/SeaquestNoFrameskip-v4_1
hyperparams:  OrderedDict([('batch_size', 256), ('clip_range', 'lin_0.1'), ('ent_coef', 0.01), ('env_wrapper', ['stable_baselines3.common.atari_wrappers.AtariWrapper']), ('frame_stack', 4), ('learning_rate', 'lin_2.5e-4'), ('n_envs', 8), ('n_epochs', 4), ('n_steps', 128), ('n_timesteps', 10000000.0), ('policy', 'CnnPolicy'), ('vf_coef', 0.5), ('normalize', False)])
env kwargs:  {}
Stacking 4 frames
Wrapping the env in a VecTransposeImage.
initial obs shape:  (1, 84, 84, 4)

Dagger iteration  0
data size:  5
[False False False False False]
benchmark acc:  1.0
agg acc:  1.0

Dagger iteration  1
data size:  3205
[False False False ... False False False]
benchmark acc:  0.6
agg acc:  0.8265210608424337
Ep rwd:  [0.]  Survived:  69
Ep rwd:  [1.]  Survived:  101
Ep rwd:  [4.]  Survived:  250
Ep rwd:  [4.]  Survived:  249
Ep


Dagger iteration  18
data size:  57605
[False False False ... False False False]
benchmark acc:  0.2
agg acc:  0.828695425744293
Ep rwd:  [7.]  Survived:  174
Ep rwd:  [21.]  Survived:  562
Ep rwd:  [22.]  Survived:  557
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [21.]  Survived:  562
Ep rwd:  [22.]  Survived:  562

Dagger iteration  19
data size:  60805
[False False False ... False False False]
benchmark acc:  0.2
agg acc:  0.8321684072033549
Ep rwd:  [14.]  Survived:  318
Ep rwd:  [21.]  Survived:  558
Ep rwd:  [22.]  Survived:  558
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  554
Ep rwd:  [14.]  Survived:  462

Dagger iteration  20
data size:  64005
[False False False ... False False False]
benchmark acc:  0.2
agg acc:  0.8377470510116397
Ep rwd:  [17.]  Survived:  349
Ep rwd:  [24.]  Survived:  562
Ep rwd:  [21.]  Survived:  551
Ep rwd:  [22.]  Survived:  560
Ep rwd:  [6.]  Survived:  348
Ep rwd:  [22.]  Survived:  560

Dagger iteration  21
data size:  67205
[False False

Ep rwd:  [21.]  Survived:  562
Ep rwd:  [21.]  Survived:  562

Dagger iteration  44
data size:  140805
[False False False ... False False False]
benchmark acc:  0.2
agg acc:  0.8826178047654558
Ep rwd:  [0.]  Survived:  20
Ep rwd:  [22.]  Survived:  558
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  561

Dagger iteration  45
data size:  144005
[False False False ... False False False]
benchmark acc:  0.2
agg acc:  0.8794416860525677
Ep rwd:  [7.]  Survived:  166
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  559
Ep rwd:  [20.]  Survived:  562
Ep rwd:  [23.]  Survived:  562

Dagger iteration  46
data size:  147205
[False False False ... False False False]
benchmark acc:  0.2
agg acc:  0.8806290547196087
Ep rwd:  [14.]  Survived:  314
Ep rwd:  [4.]  Survived:  233
Ep rwd:  [11.]  Survived:  457
Ep rwd:  [26.]  Survived:  556
Ep rwd:  [26.]  Survived:  562
Ep rwd:  [13.]  Sur

In [23]:
flip_fidelity(clf,fidelity=True)

Loading latest experiment, id=1
filter:  rl-trained-agents
algo:  ppo
env_id:  SeaquestNoFrameskip-v4
exp id:  1
log path:  rl-trained-agents/ppo/SeaquestNoFrameskip-v4_1
hyperparams:  OrderedDict([('batch_size', 256), ('clip_range', 'lin_0.1'), ('ent_coef', 0.01), ('env_wrapper', ['stable_baselines3.common.atari_wrappers.AtariWrapper']), ('frame_stack', 4), ('learning_rate', 'lin_2.5e-4'), ('n_envs', 8), ('n_epochs', 4), ('n_steps', 128), ('n_timesteps', 10000000.0), ('policy', 'CnnPolicy'), ('vf_coef', 0.5), ('normalize', False)])
env kwargs:  {}
Stacking 4 frames
Wrapping the env in a VecTransposeImage.
initial obs shape:  (1, 84, 84, 4)
Ep rwd:  [21.]  Survived:  559
Ep rwd:  [23.]  Survived:  562
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  558
Ep rwd:  [23.]  Survived:  562
Ep rwd:  [23.]  Survived:  562
Ep rwd:  [23.]  Survived:  562
Ep rwd:  [23.]  Survived:  560
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [22.]  Survived:  562
Ep rwd:  [

In [ ]:
deploy_clf(clf,fidelity=False)

In [ ]:
 r = export_text(clf, feature_names=['player_y1','cpu_y1','player_v','cpu_v','ball_x1','ball_y1', 'ball_vx','ball_vy'])
 print(r)

In [ ]:
import warnings
warnings.filterwarnings('ignore')